In [78]:
import os
import cv2
import sys
import shutil
import tensorflow as tf 
import numpy as np
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import Activation, Concatenate, Conv2DTranspose,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from skimage.io import imread, imshow
from skimage.transform import resize
from matplotlib import pyplot as plt
from imgaug import augmenters as iaa

In [ ]:
def find_files(file_path, file_name):
    pf = ""
    for root, dirs, files in os.walk(file_path):
         if file_name in files:
            pf = os.path.join(root, file_name)
            break
    return(pf)

In [ ]:
#split data on train and validation
#a SPLIT SIZE to determine the portion
#testing set has 10 images
def split_data(SOURCE_IMAGES, SOURCE_MASKS, TRAINING, TESTING, SPLIT_SIZE):
    list_samples_images = os.listdir(SOURCE_IMAGES) 
    
    num_train_samples = int(len(list_samples_images) * SPLIT_SIZE)
    for file_name in list_samples_images[:num_train_samples]:
        path_file = os.path.join(SOURCE_IMAGES, file_name)
        if os.path.getsize(path_file) != 0:
            path_mask = find_files(SOURCE_MASKS, (file_name[:-4] +'mask.png'))
            if path_mask != "":
                shutil.copyfile(path_file, os.path.join(TRAINING, 'images', file_name))            
                shutil.copyfile(path_mask, os.path.join(TRAINING, 'masks', (file_name[:-4] +'mask.png')))
    
    for file_name in list_samples_images[num_train_samples:]:
        path_file = os.path.join(SOURCE_IMAGES, file_name)
        if os.path.getsize(path_file) != 0:
            path_mask = find_files(SOURCE_MASKS, (file_name[:-4] +'mask.png'))
            if path_mask != "":
                shutil.copyfile(path_file, os.path.join(TESTING, 'images', file_name))
                shutil.copyfile(path_mask, os.path.join(TESTING, 'masks', (file_name[:-4] +'mask.png')))

SOURCE_DIR_IMAGES = "jsons/images"
SOURCE_DIR_MASKS = "jsons/mask"
TRAINING_DIR = "DataSet/train/"
TESTING_DIR = "DataSet/test/"


split_size = .9
split_data(SOURCE_DIR_IMAGES,SOURCE_DIR_MASKS, TRAINING_DIR, TESTING_DIR, split_size)

In [71]:
## SETTINGS
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
# Training, validation and testing data path
TRAINING_DIR_IMAGES = "DataSet/train/images/"
TRAINING_DIR_MASKS = "DataSet/train/masks/"
TEST_DIR_IMAGES = "DataSet/test/images/"
TEST_DIR_MASKS = "DataSet/test/masks/"

# Get the ids
train_img_ids = next(os.walk(TRAINING_DIR_IMAGES))[2]
train_mask_ids = next(os.walk(TRAINING_DIR_MASKS))[2]
test_img_ids = next(os.walk(TEST_DIR_IMAGES))[2]
test_mask_ids = next(os.walk(TEST_DIR_MASKS))[2]
# print(train_img_ids)

X_train = np.zeros((len(train_img_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_img_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
sys.stdout.flush()

for n, id_ in enumerate(train_img_ids):
    img = cv2.imread(TRAINING_DIR_IMAGES + id_)[:, :, :IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
    mask_ = cv2.imread(TRAINING_DIR_MASKS + id_[:-4] + 'mask.png') [:,:,0]
    mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
    mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Load the test data sets
X_test = np.zeros((len(test_img_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_img_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
print('Getting and resizing test images ... ')
for n, id_ in  enumerate(test_img_ids):
    img = cv2.imread(TEST_DIR_IMAGES + id_)[:, :, :IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img
    
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
    mask_ = cv2.imread(TEST_DIR_MASKS + id_[:-4]+ 'mask.png')[:,:,0]
    mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
    mask = np.maximum(mask, mask_)
    Y_test[n] = mask

print(X_train.shape)
print(X_test.shape)
print('Done!')


Getting and resizing test images ... 
(75, 512, 512, 3)
(18, 512, 512, 3)
(9, 512, 512, 3)
Done!


In [ ]:
for i in range(0,1):
    imshow(X_train[i].astype(np.uint8))
    plt.show()
    imshow(Y_train[i].astype(np.uint8))
    plt.show()

In [79]:

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_vgg16_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained VGG16 Model """
    vgg16 = VGG16(include_top=False, weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", input_tensor=inputs)

    """ Encoder """
    s1 = vgg16.get_layer("block1_conv2").output         ## (512 x 512)
    s2 = vgg16.get_layer("block2_conv2").output         ## (256 x 256)
    s3 = vgg16.get_layer("block3_conv3").output         ## (128 x 128)
    s4 = vgg16.get_layer("block4_conv3").output         ## (64 x 64)

    """ Bridge """
    b1 = vgg16.get_layer("block5_conv3").output         ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="VGG16_U-Net")
    return model


(300, 512, 512, 3)
(72, 512, 512, 3)
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
(18, 512, 512, 2)
(75, 512, 512, 2)


In [ ]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model_vgg_1 = build_vgg16_unet(input_shape)
model_vgg_1.compile(optimizer = Adam(learning_rate=0.001), loss = 'binary_crossentropy', metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=2)])

In [ ]:

checkpointer = ModelCheckpoint('model-vgg-100.h5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=6, verbose=1)
history = model_vgg_1.fit(
            X_train,
            Y_train,
            validation_split=0.2,
            batch_size=4,
            epochs=100, 
            callbacks=[checkpointer])

In [ ]:
model_vgg_1.save('model-vgg-100.h5')

for i in range(0,len(X_test)-1):
    x= X_test[i]
    x = np.expand_dims(x, axis=0)
    predict = model_vgg_1.predict(x)
    predict = np.squeeze(predict)
    predict =(predict>0.5).astype(np.uint8)
    plt.figure(figsize=(12,6))
    plt.subplot(131)
    plt.imshow(predict)
    plt.subplot(132)
    imshow(Y_test[i].astype(np.uint8))
    plt.subplot(133)
    imshow(X_test[i].astype(np.uint8))
    plt.show()

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

In [ ]:
X_train_aug = np.zeros((len(train_img_ids) * 4, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.float32)
Y_train_aug = np.zeros((len(train_img_ids) * 4, IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

for i in range(0, len(X_train)):
    X_train_aug[i * 4] = X_train[i]
    Y_train_aug[i * 4] = Y_train[i]
    X_train_aug[i * 4 + 1] = X_train[i]
    Y_train_aug[i * 4 + 1] = Y_train[i]
    X_train_aug[i * 4 + 2] = X_train[i]
    Y_train_aug[i * 4 + 2] = Y_train[i]
    X_train_aug[i * 4 + 3] = X_train[i]
    Y_train_aug[i * 4 + 3] = Y_train[i]

seq =iaa.Sequential([
    iaa.OneOf([iaa.Affine(translate_px={"x": (-10, 10), "y": (-10, 10)}), 
               iaa.Affine(rotate=(-90, 90)),
               iaa.Affine(shear=(-20, 20)),
               iaa.Crop(percent=(0, 0.15))
        ]),
    iaa.SomeOf((0,4),[
        iaa.Fliplr(1.0), # horizontally flip
        iaa.Flipud(1.0),# Vertical flip
        iaa.Add((-20,20)),
        iaa.Multiply((0.5, 1.5)),
        iaa.Sharpen(alpha=(0, 0.4), lightness=(0.5, 1.5)), # sharpen images        
        iaa.GaussianBlur(sigma=(0, 0.2)),
#         iaa.AdditiveGaussianNoise(scale=(0, 0.2*255))
    ])
], 
random_order=True # apply the augmentations in random order
)
X_train_aug, Y_train_aug = seq(images=X_train_aug, segmentation_maps=Y_train_aug)


In [80]:
model_2 = build_vgg16_unet(input_shape)
model_2.compile(optimizer = Adam(learning_rate=0.001), loss = 'binary_crossentropy', metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=2)])

In [81]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model_vgg = build_vgg16_unet(input_shape)
model_vgg.compile(optimizer = Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint('model-vgg-aug-100.h5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
history = model_2.fit(
            X_train_aug,
            Y_train_aug,
            validation_split=0.2,
            batch_size=4,
            epochs=100, 
            callbacks=[checkpointer])

Epoch 1/5
 9/20 [============>.................] - ETA: 2:06 - loss: 0.4451 - accuracy: 0.7983

In [ ]:
model_2.save('model-vgg-100.h5')
import random
for i in range(0,len(X_test)-1):
    x= X_test[i]
    x = np.expand_dims(x, axis=0)
    predict = model_2.predict(x)
    predict = np.squeeze(predict)
    predict =(predict>0.5).astype(np.uint8)
    plt.figure(figsize=(12,6))
    plt.subplot(131)
    plt.imshow(predict)
    plt.subplot(132)
    imshow(Y_test[i].astype(np.uint8))
    plt.subplot(133)
    imshow(X_test[i].astype(np.uint8))
    plt.show()